In [1]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm

output_folder = 'CovidAugust05-2022'
if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# PVI data

In [2]:
# read in fips codes
fips = pd.read_csv(
    "https://github.com/COVID19PVI/data/raw/master/Model11.2.1/data/Model_11.2.1_20200228_data.csv", 
    header=12, usecols=["name","casrn"]
).rename(columns={'name':'Name','casrn':'FIPS'})

# get the PVI data links
r = requests.get("https://github.com/COVID19PVI/data/tree/master/Model11.2.1")
soup = BeautifulSoup(r.content, 'html.parser')
links = [
    x.attrs["href"].replace("blob","raw")
    for x in soup.find_all('a', {'class':'js-navigation-open Link--primary'})[1:-1]
]

In [3]:
# combine data
all_data = []
read_from_local = False # Set to true if github isn't responding

if read_from_local:
    filenames = [link.split('/')[-1] for link in links]
    model_path = '../../data/Model11.2.1/' # PVI git repo cloned here
    for filename in tqdm(filenames):
        new_data = pd.read_csv(os.path.join(model_path, filename))
        new_data["date"] = filename.split("_")[2]
        all_data.append(new_data)

else:
    for link in tqdm(links):
        new_data = pd.read_csv("https://github.com"+link)
        new_data["date"] = link.split("_")[2]
        all_data.append(new_data)

all_data = pd.concat(all_data)
all_data["date"] = [f"{i[:4]}-{i[4:6]}-{i[6:]}" for i in all_data.date]
all_data.head()

100%|██████████| 889/889 [00:11<00:00, 75.39it/s]


,ToxPi Score,HClust Group,KMeans Group,Name,Source,Infection Rate: Transmissible Cases!25!0xcc3333ff,Infection Rate: Disease Spread!5!0xe64d4dff,Pop Concentration: Pop Mobility!10!0x57b757ff,Pop Concentration: Residential Density!10!0x5ced5cff,Intervention: Social Distancing!10!0x4258c9ff,Intervention: Testing!10!0x6079f7ff,Health & Environment: Pop Demographics!10!0x6b0b9eff,Health & Environment: Air Pollution!10!0x8e26c4ff,Health & Environment: Age Distribution!10!0x9a42c8ff,Health & Environment: Co-morbidities!10!0xb460e0ff,Health & Environment: Health Disparities!10!0xc885ecff,Health & Environment: Hospital Beds!5!0xdeb9f1ff,date
0,0.659739,1,4,"California, Solano","-121.9357,38.2683",1.000000,1.000000,0.717074,0.6905,1.000,0.504366,0.622993,0.479042,0.504195,0.236237,0.235189,0.514283,2020-02-28
1,0.624339,2,4,"California, Madera","-119.7666,37.2151",0.638652,1.000000,0.601609,0.8239,0.825,0.504366,0.605671,0.658683,0.469686,0.321223,0.590624,0.613680,2020-02-28
2,0.598916,1,4,"California, Santa Cruz","-122.0067,37.0558",0.817887,1.000000,0.704589,0.9035,1.000,0.504366,0.409785,0.269461,0.511724,0.100021,0.272557,0.531471,2020-02-28
3,0.590967,2,4,"California, Imperial","-115.3669,33.0393",0.591782,0.666667,0.585525,0.9869,0.750,0.504366,0.525758,0.628743,0.443774,0.270404,0.620412,0.516830,2020-02-28
4,0.578017,1,4,"California, Santa Clara","-121.697,37.231",0.830152,0.880000,0.793513,0.7310,1.000,0.504366,0.452019,0.353293,0.452613,0.060895,0.131427,0.461418,2020-02-28


In [4]:
drop_columns = ['Name', 'Source','date', 'ToxPi Score', 'HClust Group', 'KMeans Group', 'Hospital Beds']
for column in all_data.columns:
    column_name = column.split('!')[0]
    if ":" in column_name:
        column_name = column_name.split(": ")[1]

    if column_name in drop_columns:
        continue
    print(f"Saving {column_name}", end = "... ")
    mx = all_data.pivot(index="Name",columns="date", values=column).reset_index()
    mx = pd.merge(mx, fips, on="Name", how="left")
    mx.insert(0, "FIPS", mx.pop("FIPS"))
    mx.to_csv(f"{output_folder}/{column_name}.csv", index=False, float_format='%.4f')
    print(u'\u2705')

Saving Transmissible Cases... ✅
Saving Disease Spread... ✅
Saving Pop Mobility... ✅
Saving Residential Density... ✅
Saving Social Distancing... ✅
Saving Testing... ✅
Saving Pop Demographics... ✅
Saving Air Pollution... ✅
Saving Age Distribution... ✅
Saving Co-morbidities... ✅
Saving Health Disparities... ✅


In [5]:
all_data.columns

Index(['ToxPi Score', 'HClust Group', 'KMeans Group', 'Name', 'Source',
       'Infection Rate: Transmissible Cases!25!0xcc3333ff',
       'Infection Rate: Disease Spread!5!0xe64d4dff',
       'Pop Concentration: Pop Mobility!10!0x57b757ff',
       'Pop Concentration: Residential Density!10!0x5ced5cff',
       'Intervention: Social Distancing!10!0x4258c9ff',
       'Intervention: Testing!10!0x6079f7ff',
       'Health & Environment: Pop Demographics!10!0x6b0b9eff',
       'Health & Environment: Air Pollution!10!0x8e26c4ff',
       'Health & Environment: Age Distribution!10!0x9a42c8ff',
       'Health & Environment: Co-morbidities!10!0xb460e0ff',
       'Health & Environment: Health Disparities!10!0xc885ecff',
       'Health & Environment: Hospital Beds!5!0xdeb9f1ff', 'date'],
      dtype='object')

# Cases
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22.

In [6]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv")

file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-07-24,2022-07-25,2022-07-26,2022-07-27,2022-07-28,2022-07-29,2022-07-30,2022-07-31,2022-08-01,2022-08-02
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,17268,17349,17392,17425,17455,17468,17468,17468,17468,17468
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,61268,61519,61632,61756,61851,61967,61967,61967,61967,61967
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,6205,6244,6258,6276,6293,6308,6308,6308,6308,6308
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6882,6909,6921,6930,6937,6947,6947,6947,6947,6947


In [7]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Cases.csv', index=False)

# Deaths
Source [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). Data collection starts from 2020-01-22.

In [8]:
response = requests.get("https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object)
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-07-24,2022-07-25,2022-07-26,2022-07-27,2022-07-28,2022-07-29,2022-07-30,2022-07-31,2022-08-01,2022-08-02
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,220,220,220,220,220,220,220,220,220,220
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,686,687,687,687,687,687,687,687,687,687
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,100,100,100,100,100,100,100,100,100,100
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,105,105,105,105,105,105,105,105,105,105


In [9]:
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)

df = df[(~df['FIPS'].isna()) & (df['FIPS']>0)]
df.fillna(0, inplace=True)
df.to_csv(f'{output_folder}/Deaths.csv', index=False)

# Vaccination
From [CDC vaccinations in the US counties](https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh). Data collection starts from 2020-12-13.

In [10]:
response = requests.get("https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD")
file_object = io.StringIO(response.content.decode('utf-8'))
df = pd.read_csv(file_object, low_memory=False)
df.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_5Plus,Administered_Dose1_Recip_5PlusPop_Pct,...,Booster_Doses_Vax_Pct_UR_Equity,Booster_Doses_12PlusVax_Pct_UR_Equity,Booster_Doses_18PlusVax_Pct_UR_Equity,Booster_Doses_65PlusVax_Pct_UR_Equity,Census2019,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,08/03/2022,54025,31,Greenbrier County,WV,97.2,21686.0,62.6,21644.0,65.8,...,7.0,8.0,8.0,7.0,34662.0,32879.0,4979.0,30238.0,27900.0,8136.0
1,08/03/2022,17043,31,DuPage County,IL,98.3,776606.0,84.1,771299.0,88.7,...,4.0,4.0,4.0,4.0,922921.0,869134.0,153791.0,789146.0,715343.0,148998.0
2,08/03/2022,01017,31,Chambers County,AL,92.5,12875.0,38.7,12867.0,41.0,...,6.0,6.0,7.0,6.0,33254.0,31372.0,5031.0,28624.0,26341.0,6715.0
3,08/03/2022,36007,31,Broome County,NY,97.6,132779.0,69.7,132528.0,73.4,...,4.0,4.0,4.0,3.0,190488.0,180602.0,26919.0,166375.0,153683.0,36980.0
4,08/03/2022,16023,31,Butte County,ID,97.7,1424.0,54.8,1424.0,58.1,...,2.0,3.0,3.0,2.0,2597.0,2450.0,484.0,2204.0,1966.0,614.0


In [11]:
df = df[df['FIPS']!='UNK']
df['FIPS'] = df['FIPS'].astype(int)
vaccination_columns = ['Administered_Dose1_Pop_Pct','Series_Complete_Pop_Pct']
df = df[['Date', 'FIPS'] + vaccination_columns]

In [12]:
df['Date'] = pd.to_datetime(df['Date'])
# df = df.sort_values(by=['FIPS', 'Date']).reset_index(drop=True)
df.fillna(0, inplace=True)

In [13]:
vaccinationOneDose = df[['Date', 'FIPS', 'Administered_Dose1_Pop_Pct']].pivot_table(
    values='Administered_Dose1_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
vaccinationOneDose.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,...,2022-06-14 00:00:00,2022-06-15 00:00:00,2022-06-16 00:00:00,2022-06-22 00:00:00,2022-06-29 00:00:00,2022-07-06 00:00:00,2022-07-13 00:00:00,2022-07-20 00:00:00,2022-07-27 00:00:00,2022-08-03 00:00:00
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56.1,56.1,56.1,56.2,56.3,56.4,56.4,56.5,56.7,56.8
1,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,64.9,64.9,64.9,65.0,65.1,65.1,65.2,65.3,65.4,65.5
2,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,56.2,56.2,56.2,56.2,56.3,56.3,56.4,56.5,56.6,56.6
3,1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.7,42.7,42.7,42.8,42.9,42.9,42.9,43.0,43.0,43.1
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.7,38.7,38.7,38.7,38.8,38.8,38.9,38.9,39.0,39.0


In [14]:
vaccinationCompleteDose = df[['Date', 'FIPS', 'Series_Complete_Pop_Pct']].pivot_table(
    values='Series_Complete_Pop_Pct',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)
vaccinationCompleteDose.head()

,FIPS,2020-12-13 00:00:00,2020-12-14 00:00:00,2020-12-15 00:00:00,2020-12-16 00:00:00,2020-12-17 00:00:00,2020-12-18 00:00:00,2020-12-19 00:00:00,2020-12-20 00:00:00,2020-12-21 00:00:00,...,2022-06-14 00:00:00,2022-06-15 00:00:00,2022-06-16 00:00:00,2022-06-22 00:00:00,2022-06-29 00:00:00,2022-07-06 00:00:00,2022-07-13 00:00:00,2022-07-20 00:00:00,2022-07-27 00:00:00,2022-08-03 00:00:00
0,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.6,44.6,44.7,44.7,44.4,44.5,44.5,44.6,44.7,44.8
1,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,51.5,51.5,51.5,51.5,51.6,51.6,51.7,51.8,51.9,51.9
2,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.9,45.9,45.9,45.9,46.0,46.0,46.1,46.2,46.3,46.3
3,1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.0,35.0,35.0,35.0,35.0,35.1,35.2,35.3,35.4,35.5
4,1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.0,32.0,32.0,32.0,32.0,32.1,32.1,32.1,32.2,32.2


In [15]:
vaccinationCompleteDose.to_csv(f'{output_folder}/Vaccination.csv', index=False)
vaccinationOneDose.to_csv(f'{output_folder}/VaccinationOneDose.csv', index=False)